In [ ]:
%matplotlib widget
import batoid_rubin
import ipywidgets

In [ ]:
debug = ipywidgets.Output()
debug
# debug=None

In [ ]:
app = batoid_rubin.align_game.AlignGame(debug=debug, nthread=1)
app.display()